In [2]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn
!pip3 install tk

     |████████████████████████████████| 10.3 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 58.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 64.9 MB/s eta 0:00:01
     |████████████████████████████████| 27.4 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 66.1 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 184 kB 4.2 MB/s eta 0:00:01
  Created wheel for spectral: filename=spectral-0.22.2-py3-none-any.whl size=212936 sha256=b24ac41e01b7de09c266a043fda6c03bc5cf7d12c0570fb455bbbdfdfea82ab9
  Stored in directory: /home/jovyan/.cache/pip/wheels/8b/3c/53/b5300d56bee75c16de9b19cd481c419cc63929f2e4019a6235
Successfully built spectral
     |████████████████████████████████| 285 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 29.2 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 77.0 MB/s eta 0:00:01
     |████████

     |████████████████████████████████| 57 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 47.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 49.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 41.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 84.4 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 73.1 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 136 kB 77.2 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 68.3 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 9.0 MB/s  eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=41100dbabee9fc72b8

In [3]:
%matplotlib inline
import spectral as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from spectrum import *
from spectrum_ml import *
from ipywidgets import interact, interactive, widgets
from IPython.display import display
import os, os.path, io
from tkinter import Button
from tkinter import Label
from tkinter import StringVar
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from scipy.stats import rankdata
import sklearn
from keras.layers import Input, Dense, Conv1D, Conv2DTranspose,Lambda,Flatten,Reshape
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.utils import plot_model
from keras.losses import mse, binary_crossentropy
from keras.losses import MAPE, cosine_similarity, MSLE, mae
import tensorflow as tf
from sklearn import metrics,svm
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib

In [10]:
!pip3 install -U tensorflow

In [11]:
!pip3 install -U sklearn

In [4]:
def on_button_clicked1(b):
    with output1:
        output1.clear_output()
        
        os.environ['KMP_DUPLICATE_LIB_OK']='True'
        print("Environnement Chargé")
        

In [5]:
button1 = widgets.Button(description="Rechargez l'environnement")
output1 = widgets.Output()
display(button1, output1)

button1.on_click(on_button_clicked1)

Button(description="Rechargez l'environnement", style=ButtonStyle())

Output()

In [6]:
wl_bis = widgets.FileUpload(
    accept='', 
    multiple=False,  
    description = 'Fichier CSV'
    )

display(wl_bis)

FileUpload(value={}, description='Fichier CSV')

In [12]:
def on_button_clicked2(b):
    with output2:
        output2.clear_output()
            
        loaded_model = tf.keras.models.model_from_json(open('./train_models/loaded_model_v2_sqrtmse_loss.json').read())
        loaded_model.load_weights("./train_models/loaded_model_v2_sqrtmse_loss.h5")
        loaded_model.compile(optimizer='RMSprop')
        SVM_MSE = joblib.load('./train_models/SVM_spectral_classifier_MSE.pkl')
            
        X = pd.read_csv("./train_models/average_profile_no_outlier.csv",index_col=[0,1,2])
        X = X.iloc[np.random.randint(0,X.shape[0],300),:]
        X += np.random.randn(*X.shape)*0.01
        neg = pd.read_csv("./train_models/spectrum_avg_norm.csv",index_col=0).iloc[:3,:]
        neg = pd.concat((neg,pd.read_csv("./train_models/feuille_spectrum_avg_norm.csv",index_col=0)))
        neg = neg.iloc[np.random.randint(0,3,300),:]
        neg += np.random.randn(*neg.shape)*0.01
        #Spectre_test = pd.read_csv("test_spectrum_avg_norm.csv",index_col=0)
        Spectre_test = pd.read_csv(io.BytesIO(wl_bis.value[list(wl_bis.value.keys())[0]]['content']), index_col=0)
        #Spectre_test
            
        X_test = X
        decoded_X_test = loaded_model.predict(X)
        decoded_neg = loaded_model.predict(neg)
        decoded_Spectre_test = loaded_model.predict(Spectre_test)
            
        #plt.plot(decoded_X.T)
        plt.plot(decoded_Spectre_test.T, "b")
        plt.plot(Spectre_test.T, "red")
        #plt.plot(neg[:20].T,"green", alpha = 0.2)
        #plt.plot(decoded_neg[:20].T , "black", alpha = 0.2)
        plt.show()
            
        lim_inf = 0
        lim_sup = 448

        """
        mse_feuille = np.array(mse(decoded_neg,neg.iloc))
        mse_puceron = np.array(mse(decoded_X, X.iloc))
        mse_Spectre = np.array(mse(decoded_Spectre_test, Spectre_test.iloc))
        """

        mse_feuille = np.array(mse(decoded_neg[:,lim_inf:lim_sup],neg.iloc[:,lim_inf:lim_sup]))
        mse_puceron = np.array(mse(decoded_X_test[:,lim_inf:lim_sup], X_test.iloc[:,lim_inf:lim_sup]))
        mse_Spectre = np.array(mse(decoded_Spectre_test[:,lim_inf:lim_sup], Spectre_test.iloc[:,lim_inf:lim_sup]))


        sqrt_mse_p = np.array(np.sqrt(mse(decoded_X_test[:,lim_inf:lim_sup], X_test.iloc[:,lim_inf:lim_sup])))
        sqrt_mse_f = np.array(np.sqrt(mse(decoded_neg[:,lim_inf:lim_sup], neg.iloc[:,lim_inf:lim_sup])))
        sqrt_Spectre = np.array(np.sqrt(mse(decoded_Spectre_test[:,lim_inf:lim_sup], Spectre_test.iloc[:,lim_inf:lim_sup])))

        mape_feuille = np.array(MAPE(decoded_neg[:,lim_inf:lim_sup], neg.iloc[:,lim_inf:lim_sup]))
        mape_puceron = np.array(MAPE(decoded_X_test[:,lim_inf:lim_sup], X_test.iloc[:,lim_inf:lim_sup]))
        mape_Spectre = np.array(MAPE(decoded_Spectre_test[:,lim_inf:lim_sup], Spectre_test.iloc[:,lim_inf:lim_sup]))

        mselog_feuille = np.array(MSLE(decoded_neg[:,lim_inf:lim_sup], neg.iloc[:,lim_inf:lim_sup]))
        mselog_puceron = np.array(MSLE(decoded_X_test[:,lim_inf:lim_sup], X_test.iloc[:,lim_inf:lim_sup]))
        mselog_Spectre = np.array(MSLE(decoded_Spectre_test[:,lim_inf:lim_sup], Spectre_test.iloc[:,lim_inf:lim_sup]))
            
        bins = np.linspace(-0.001, 0.1, 150)

        plt.hist(mse_feuille, bins, alpha=0.5, label='feuille')
        plt.hist(mse_puceron, bins, alpha=0.5, label='puceron')
        plt.hist(mse_Spectre, bins, alpha=1, label = "Spectre")
        plt.ylim(-0.1,6)
        plt.legend(loc='upper left')
        plt.title("MSE")
        plt.show()
            
        bins = np.linspace(-0.0, 0.3, 150)

        plt.hist(sqrt_mse_f, bins, alpha=0.5, label='feuille')
        plt.hist(sqrt_mse_p, bins, alpha=0.5, label='puceron')
        plt.hist(sqrt_Spectre, bins, alpha=1, label = "Spectre")
        plt.ylim(-0.1,6)
        plt.legend(loc='upper left')
        plt.title("SQRT(MSE)")
        plt.show()
            
        bins = np.linspace(-0.001, 0.003, 100)

        plt.hist(mselog_feuille, bins, alpha=0.5, label='feuille')
        plt.hist(mselog_puceron, bins, alpha=0.5, label='puceron')
        plt.hist(mselog_Spectre, bins, alpha=1, label = "Spectre")
        plt.ylim(-1,10)
        plt.legend(loc='upper left')
        plt.title("Log(MSE)")
        plt.show()
            
        bins = np.linspace(-0.01, 100, 150)

        plt.hist(mape_feuille, bins, alpha=0.5, label='feuille')
        plt.hist(mape_puceron, bins, alpha=0.5, label='puceron')
        plt.hist(mape_Spectre, bins, alpha=1, label = "Spectre")
        plt.legend(loc='upper left')
        plt.title("MAPE")
        plt.show()
            
        print(Spectre_test)
        print(mse_Spectre)
        PREDICTION_SPECTRE = SVM_MSE.predict(np.array(mse_Spectre).reshape(-1,1))
        print("\n Prediction du spectre :")
        list_pred = []
        for i,e in enumerate(Spectre_test.index):
            list_pred.append((str(e)+" : "+str(PREDICTION_SPECTRE[i])))
        #print(Spectre_test.index)
        #print(PREDICTION_SPECTRE)
        print(list_pred)
    
    

In [13]:
button2 = widgets.Button(description="VAE")
output2 = widgets.Output()
display(button2, output2)

button2.on_click(on_button_clicked2)

Button(description='VAE', style=ButtonStyle())

Output()